# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities = cities.drop(['NFL','MLB','NBA'],axis=1)
    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]
        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*', team)
            return lst 
    cities['NHL'] = cities['NHL'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NHL')

    nhl_df = nhl_df[nhl_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*','')
        lst = team.split()
        if team.startswith('Toronto') or team.startswith('Detroit') or team.startswith('Columbus') or team.startswith('Vegas'):
            return (lst[1]+' '+lst[2])
        else:
            return lst[-1]
    nhl_df['team']  = nhl_df['team'].map(clean_team_wl)  
    nhl_df = nhl_df[['team','W','L']]

    joint = pd.merge(cities_explode,nhl_df,left_on='NHL',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()
    
    population_by_region = wl_result['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = wl_result['wlratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    x,y = stats.pearsonr(population_by_region, win_loss_by_region)
    return x

In [17]:
def nhl_df(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities = cities.drop(['NFL','MLB','NBA'],axis=1)
    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]
        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*', team)
            return lst 
    cities['NHL'] = cities['NHL'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NHL')

    nhl_df = nhl_df[nhl_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*','')
        lst = team.split()
        if team.startswith('Toronto') or team.startswith('Detroit') or team.startswith('Columbus') or team.startswith('Vegas'):
            return (lst[1]+' '+lst[2])
        else:
            return lst[-1]
    nhl_df['team']  = nhl_df['team'].map(clean_team_wl)  
    nhl_df = nhl_df[['team','W','L']]

    joint = pd.merge(cities_explode,nhl_df,left_on='NHL',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

    return wl_result

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['NFL','MLB','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]
        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
            return lst 
    cities['NBA'] = cities['NBA'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NBA')

    nba_df = nba_df[nba_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*',' ')
        lst = team.split()
        if team.startswith('Portland'):
            return (lst[1]+' '+lst[2])
        elif team.startswith('New York') or team.startswith('Golden State') or team.startswith('Oklahoma City') or team.startswith('New Orleans') or team.startswith('San Antonio') or team.startswith('Los Angeles') :
            return (lst[2])
        else:
            return lst[1]
    nba_df['team']  = nba_df['team'].map(clean_team_wl)  
    nba_df = nba_df[['team','W','L']]

    joint = pd.merge(cities_explode,nba_df,left_on='NBA',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()
    
    population_by_region = wl_result['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = wl_result['wlratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    x,y = stats.pearsonr(population_by_region, win_loss_by_region)
    return x

In [42]:
def nba_df():
    # YOUR CODE HERE
    #raise NotImplementedError()
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['NFL','MLB','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]
        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
            return lst 
    cities['NBA'] = cities['NBA'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NBA')

    nba_df = nba_df[nba_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*',' ')
        lst = team.split()
        if team.startswith('Portland'):
            return (lst[1]+' '+lst[2])
        elif team.startswith('New York') or team.startswith('Golden State') or team.startswith('Oklahoma City') or team.startswith('New Orleans') or team.startswith('San Antonio') or team.startswith('Los Angeles') :
            return (lst[2])
        else:
            return lst[1]
    nba_df['team']  = nba_df['team'].map(clean_team_wl)  
    nba_df = nba_df[['team','W','L']]

    joint = pd.merge(cities_explode,nba_df,left_on='NBA',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()
    
    return wl_result

nba_df()

,wlratio,Population
Metropolitan area,,
Atlanta,0.292683,5789700
Boston,0.670732,4794447
Charlotte,0.439024,2474314
Chicago,0.329268,9512999
Cleveland,0.609756,2055612
Dallas–Fort Worth,0.292683,7233323
Denver,0.560976,2853077
Detroit,0.475610,4297617
Houston,0.792683,6772470


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['NFL','NBA','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]

        lst = []
        if team =='':
            return None
        else: 
            lst = re.findall('[A-Z][^A-Z]*', team)
            for i, item in enumerate(lst):
                if item.endswith(' '):
                    lst[i] = lst[i] + lst[i+1]
                    lst.pop(i+1)
            return lst 
    cities['MLB'] = cities['MLB'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('MLB')

    mlb_df = mlb_df[mlb_df.year==2018]
    def clean_team_wl(team):
        lst = team.split()
        if team.startswith('Boston') or team.startswith('Toronto') or team.startswith('Chicago White') :
            return (lst[1]+' '+lst[2])
        else:
            return lst[-1]
    mlb_df['team']  = mlb_df['team'].apply(clean_team_wl)  
    mlb_df = mlb_df[['team','W','L']]

    joint = pd.merge(cities_explode,mlb_df,left_on='MLB',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()
    
    population_by_region = wl_result['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = wl_result['wlratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    x,y = stats.pearsonr(population_by_region, win_loss_by_region)
    return x

In [20]:
def mlb_df(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['NFL','NBA','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('—',"")
        team = team.split('[')[0]

        lst = []
        if team =='':
            return None
        else: 
            lst = re.findall('[A-Z][^A-Z]*', team)
            for i, item in enumerate(lst):
                if item.endswith(' '):
                    lst[i] = lst[i] + lst[i+1]
                    lst.pop(i+1)
            return lst 
    cities['MLB'] = cities['MLB'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('MLB')

    mlb_df = mlb_df[mlb_df.year==2018]
    def clean_team_wl(team):
        lst = team.split()
        if team.startswith('Boston') or team.startswith('Toronto') or team.startswith('Chicago White') :
            return (lst[1]+' '+lst[2])
        else:
            return lst[-1]
    mlb_df['team']  = mlb_df['team'].apply(clean_team_wl)  
    mlb_df = mlb_df[['team','W','L']]

    joint = pd.merge(cities_explode,mlb_df,left_on='MLB',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()
    
    return wl_result

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['MLB','NBA','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('— ',"")
        team = team.replace('—',"")
        team = team.split('[')[0]

        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
            return lst 
    cities['NFL'] = cities['NFL'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NFL')

    nfl_df = nfl_df[nfl_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*','')
        team = team.replace('+','')
        lst = team.split()
        return lst[-1]
    nfl_df['team']  = nfl_df['team'].map(clean_team_wl)  
    nfl_df = nfl_df[['team','W','L']]

    joint = pd.merge(cities_explode,nfl_df,left_on='NFL',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

    
    population_by_region = wl_result['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = wl_result['wlratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    x,y = stats.pearsonr(population_by_region, win_loss_by_region)
    return x

In [21]:
def nfl_df(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.drop(['MLB','NBA','NHL'],axis=1)

    def clean_team_pop(team):
        team = team.replace('— ',"")
        team = team.replace('—',"")
        team = team.split('[')[0]

        have_space = False
        for i in team:
            if i ==' ':
                have_space = True
        lst = []
        if team =='':
            return None
        elif have_space:
            lst.append(team)
            return lst
        else: 
            lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
            return lst 
    cities['NFL'] = cities['NFL'].map(clean_team_pop)
    cities = cities.dropna()
    cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
    cities['Population'] = cities['Population'].astype(int)
    cities_explode = cities.explode('NFL')

    nfl_df = nfl_df[nfl_df.year==2018]
    def clean_team_wl(team):
        team = team.replace('*','')
        team = team.replace('+','')
        lst = team.split()
        return lst[-1]
    nfl_df['team']  = nfl_df['team'].map(clean_team_wl)  
    nfl_df = nfl_df[['team','W','L']]

    joint = pd.merge(cities_explode,nfl_df,left_on='NFL',right_on='team',how='inner')
    joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
    wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

    return wl_result

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    def nhl_df(): 
        nhl_df=pd.read_csv("assets/nhl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]

        cities = cities.drop(['NFL','MLB','NBA'],axis=1)
        def clean_team_pop(team):
            team = team.replace('—',"")
            team = team.split('[')[0]
            have_space = False
            for i in team:
                if i ==' ':
                    have_space = True
            lst = []
            if team =='':
                return None
            elif have_space:
                lst.append(team)
                return lst
            else: 
                lst = re.findall('[A-Z][^A-Z]*', team)
                return lst 
        cities['NHL'] = cities['NHL'].map(clean_team_pop)
        cities = cities.dropna()
        cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
        cities['Population'] = cities['Population'].astype(int)
        cities_explode = cities.explode('NHL')

        nhl_df = nhl_df[nhl_df.year==2018]
        def clean_team_wl(team):
            team = team.replace('*','')
            lst = team.split()
            if team.startswith('Toronto') or team.startswith('Detroit') or team.startswith('Columbus') or team.startswith('Vegas'):
                return (lst[1]+' '+lst[2])
            else:
                return lst[-1]
        nhl_df['team']  = nhl_df['team'].map(clean_team_wl)  
        nhl_df = nhl_df[['team','W','L']]

        joint = pd.merge(cities_explode,nhl_df,left_on='NHL',right_on='team',how='inner')
        joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
        wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

        return wl_result
    
    def nba_df():
        nba_df=pd.read_csv("assets/nba.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.drop(['NFL','MLB','NHL'],axis=1)

        def clean_team_pop(team):
            team = team.replace('—',"")
            team = team.split('[')[0]
            have_space = False
            for i in team:
                if i ==' ':
                    have_space = True
            lst = []
            if team =='':
                return None
            elif have_space:
                lst.append(team)
                return lst
            else: 
                lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
                return lst 
        cities['NBA'] = cities['NBA'].map(clean_team_pop)
        cities = cities.dropna()
        cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
        cities['Population'] = cities['Population'].astype(int)
        cities_explode = cities.explode('NBA')

        nba_df = nba_df[nba_df.year==2018]
        def clean_team_wl(team):
            team = team.replace('*',' ')
            lst = team.split()
            if team.startswith('Portland'):
                return (lst[1]+' '+lst[2])
            elif team.startswith('New York') or team.startswith('Golden State') or team.startswith('Oklahoma City') or team.startswith('New Orleans') or team.startswith('San Antonio') or team.startswith('Los Angeles') :
                return (lst[2])
            else:
                return lst[1]
        nba_df['team']  = nba_df['team'].map(clean_team_wl)  
        nba_df = nba_df[['team','W','L']]

        joint = pd.merge(cities_explode,nba_df,left_on='NBA',right_on='team',how='inner')
        joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
        wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

        return wl_result

    def mlb_df(): 
        mlb_df=pd.read_csv("assets/mlb.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.drop(['NFL','NBA','NHL'],axis=1)

        def clean_team_pop(team):
            team = team.replace('—',"")
            team = team.split('[')[0]

            lst = []
            if team =='':
                return None
            else: 
                lst = re.findall('[A-Z][^A-Z]*', team)
                for i, item in enumerate(lst):
                    if item.endswith(' '):
                        lst[i] = lst[i] + lst[i+1]
                        lst.pop(i+1)
                return lst 
        cities['MLB'] = cities['MLB'].map(clean_team_pop)
        cities = cities.dropna()
        cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
        cities['Population'] = cities['Population'].astype(int)
        cities_explode = cities.explode('MLB')

        mlb_df = mlb_df[mlb_df.year==2018]
        def clean_team_wl(team):
            lst = team.split()
            if team.startswith('Boston') or team.startswith('Toronto') or team.startswith('Chicago White') :
                return (lst[1]+' '+lst[2])
            else:
                return lst[-1]
        mlb_df['team']  = mlb_df['team'].apply(clean_team_wl)  
        mlb_df = mlb_df[['team','W','L']]

        joint = pd.merge(cities_explode,mlb_df,left_on='MLB',right_on='team',how='inner')
        joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
        wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

        return wl_result

    def nfl_df(): 
        nfl_df=pd.read_csv("assets/nfl.csv")
        cities=pd.read_html("assets/wikipedia_data.html")[1]
        cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        cities = cities.drop(['MLB','NBA','NHL'],axis=1)

        def clean_team_pop(team):
            team = team.replace('— ',"")
            team = team.replace('—',"")
            team = team.split('[')[0]

            have_space = False
            for i in team:
                if i ==' ':
                    have_space = True
            lst = []
            if team =='':
                return None
            elif have_space:
                lst.append(team)
                return lst
            else: 
                lst = re.findall('[A-Z][^A-Z]*|[0-9][0-9][a-z]*', team)
                return lst 
        cities['NFL'] = cities['NFL'].map(clean_team_pop)
        cities = cities.dropna()
        cities.rename(columns = {'Population (2016 est.)[8]':'Population'},inplace=True)
        cities['Population'] = cities['Population'].astype(int)
        cities_explode = cities.explode('NFL')

        nfl_df = nfl_df[nfl_df.year==2018]
        def clean_team_wl(team):
            team = team.replace('*','')
            team = team.replace('+','')
            lst = team.split()
            return lst[-1]
        nfl_df['team']  = nfl_df['team'].map(clean_team_wl)  
        nfl_df = nfl_df[['team','W','L']]

        joint = pd.merge(cities_explode,nfl_df,left_on='NFL',right_on='team',how='inner')
        joint['wlratio'] = joint['W'].astype(float)/(joint['W'].astype(float)+joint['L'].astype(float))
        wl_result = joint.groupby(joint['Metropolitan area'])['wlratio','Population'].mean()

        return wl_result
    
    df_nhl = nhl_df()
    df_nba = nba_df()
    df_mlb = mlb_df()
    df_nfl = nfl_df()
    
    def pval(dfx,dfy):
        joint = pd.merge(dfx, dfy, left_index=True, right_index=True, how='inner')
        joint = joint.dropna()
        ts,pval = stats.ttest_rel(joint['wlratio_x'],joint['wlratio_y'])
        return pval

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    df_lst = [df_nfl,df_nba,df_nhl,df_mlb] 

    for i,item in enumerate(df_lst):
        p_values.loc[sports[i]] = [pval(item,item2) for item2 in df_lst]
        
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

df_nhl = nhl_df()
df_nba = nba_df()
df_mlb = mlb_df()
df_nfl = nfl_df()

def pval(dfx,dfy):
    joint = pd.merge(dfx, dfy, left_index=True, right_index=True, how='inner')
    joint = joint.dropna()
    ts,pval = stats.ttest_rel(joint['wlratio_x'],joint['wlratio_y'])
    return pval

sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

df_lst = [df_nfl,df_nba,df_nhl,df_mlb] 

for i,item in enumerate(df_lst):
    p_values.loc[sports[i]] = [pval(item,item2) for item2 in df_lst]

p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
